## LLL-алгоритм редукции базиса решётки

LLL-алгоритм последовательно редуцирует базис решётки, стремясь сделать векторы базиса как можно короче при сохранении целочисленности линейных комбинаций.

### Основные этапы алгоритма

1.  **Ортогонализация Грама–Шмидта:**
    Для текущего базиса вычисляются ортогональные векторы $b_i^*$ и коэффициенты проекции $\mu_{i,j}$:
    $$\mu_{i,j} = \frac{(b_i, b_j^*)}{(b_j^*, b_j^*)}$$

2.  **Уменьшение (Size Reduction):**
    Для каждого вектора $b_k$ выполняются вычитания целочисленных кратных предыдущих векторов так, чтобы все коэффициенты $\mu_{k,j}$ были близки к нулю:
    $$|\mu_{k,j}| \le 0.5$$

3.  **Проверка условия Ловаса:**
    Если квадрат нормы текущего ортогонального вектора $|b_k^*|^2$ меньше чем $(\delta - \mu_{k,k-1}^2)|b_{k-1}^*|^2$, где $\delta$ - параметр (обычно $\delta = 0.75$), то векторы $b_k$ и $b_{k-1}$ меняются местами, и индекс $k$ уменьшается. В противном случае индекс $k$ увеличивается.

4.  **Повторение шагов:**
    Шаги 1-3 повторяются до тех пор, пока для всего базиса не будет выполнено условие Ловаса. В этот момент базис считается «LLL-редуцированным».

### Свойства LLL-редуцированного базиса

LLL-редукция гарантирует, что полученные векторы базиса:

* Почти ортогональны.
* Достаточно коротки.

In [39]:
import numpy as np

def gram_schmidt(B):
    B = np.array(B, dtype=float)
    n = B.shape[0]
    m = B.shape[1]
    B_star = np.zeros((n, m))
    mu = np.zeros((n, n))
    B_norm = np.zeros(n)
    for i in range(n):
        v = B[i].copy()
        for j in range(i):
            mu[i, j] = np.dot(v, B_star[j]) / B_norm[j]
            v = v - mu[i, j] * B_star[j]
        B_star[i] = v
        B_norm[i] = np.dot(v, v)
    return B_star, mu, B_norm

def lll_reduction(B, delta=0.75):
    B = np.array(B, dtype=float)
    n = B.shape[0]
    B_star, mu, B_norm = gram_schmidt(B)
    k = 1
    while k < n:
        for j in range(k-1, -1, -1):
            q = round(mu[k, j])
            if q != 0:
                B[k] -= q * B[j]
        B_star, mu, B_norm = gram_schmidt(B)
        if B_norm[k] < (delta - mu[k, k-1]**2) * B_norm[k-1]:
            B[[k, k-1]] = B[[k-1, k]]
            k = max(k-1, 1)
            B_star, mu, B_norm = gram_schmidt(B)
        else:
            k += 1
    return B


In [40]:
basis = [[1, 1, 1], [-1, 0, 2], [3, 5, 6]]  # базис матрицы (столбцами)
reduced = lll_reduction(basis, delta=0.75)
print(reduced)



[[ 0.  1.  0.]
 [ 1.  0.  1.]
 [-1.  0.  2.]]
